In [1]:
import numpy as np
import pandas as pd

In [16]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [17]:
#util function to check for ints
def check_int(value):
    try:
        int(value)
        return np.NaN
    except ValueError:
        return value

In [18]:
df=pd.read_parquet("../data/transformed_data.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 717 entries, 19325173 to 16998076
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   coordinate                       717 non-null    object 
 1   address                          717 non-null    object 
 2   location_name                    717 non-null    object 
 3   typeSummary                      717 non-null    object 
 4   ongoing_bidding                  717 non-null    bool   
 5   foreclosure                      717 non-null    bool   
 6   price                            716 non-null    object 
 7   price_per_area                   702 non-null    object 
 8   has_price_change                 717 non-null    bool   
 9   rooms                            714 non-null    object 
 10  fee                              706 non-null    object 
 11  age                              717 non-null    int64  
 12  deactivate

In [19]:
df2 = df.copy()

In [20]:
df2

,coordinate,address,location_name,typeSummary,ongoing_bidding,foreclosure,price,price_per_area,has_price_change,rooms,fee,age,deactivated_before_open_house,small_image_url,medium_image_url,url,state_labels,visible_project_units,broker_agency_name,locations_string,should_highlight_showings,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus,price_change,area,date,balcony,elevator,patio,tenure_owned,living_area,supplemental_area
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
19325173,"[59.3239555, 18.0685062]",Lilla Nygatan 11,Stockholms kommun,Bostadsrättslägenhet,False,False,6 795 000 kr,106 172 kr/m²,False,3 rum,5 850 kr/mån,36,False,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-3rum-gam...,[],0,HusmanHagberg Södermalm/Gamla Stan,"Gamla Stan, Stockholms kommun",False,19325173,common_listing,premium,active,for_sale,NaN,gamla_stan,20221217,False,False,False,False,64 m²,None
19102415,"[59.32596899944557, 18.074948653244743]",Skeppsbron 6 / Skeppar Karls Gränd 2A,Stockholms kommun,Bostadsrättslägenhet,False,False,14 995 000 kr,99 305 kr/m²,False,4 rum,7 909 kr/mån,40,False,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-4rum-gam...,[],0,Ambassadör Fastighetsmäkleri AB,"Gamla Stan, Stockholms kommun",False,19102415,common_listing,premium,active,for_sale,NaN,gamla_stan,20221217,False,False,False,False,151 m²,None
19158120,"[59.32439338293433, 18.068599264461106]",Stora Nygatan 22,Stockholms kommun,Bostadsrättslägenhet,False,False,19 800 000 kr,113 143 kr/m²,True,6 rum,7 999 kr/mån,78,False,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-6rum-gam...,[],0,Eklund Stockholm New York,"Gamla stan, Stockholms kommun",False,19158120,common_listing,plus,active,for_sale,-10.0,gamla_stan,20221217,False,False,False,False,175 m²,None
18994274,"[59.32494731233097, 18.073308919077213]",Baggensgatan 12,Stockholms kommun,"Lägenhet, andel i bostadsförening",True,False,11 895 000 kr,143 313 kr/m²,False,4 rum,1 kr/mån,119,False,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-4rum-gam...,"[{'category': 'state', 'identifier': 'ongoing_...",0,Wallenberg Fastighetsförmedling,"Gamla Stan, Stockholms kommun",False,18994274,common_listing,basic,active,for_sale,NaN,gamla_stan,20221217,False,False,False,False,83 m²,None
18953936,"[59.32418950642289, 18.073433864314342]",Baggensgatan 21,Stockholms kommun,Bostadsrättslägenhet,True,False,19 500 000 kr,96 059 kr/m²,True,4 rum,8 270 kr/mån,131,False,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-4rum-gam...,"[{'category': 'state', 'identifier': 'ongoing_...",0,Eklund Stockholm New York,"Gamla stan, Stockholms kommun",False,18953936,common_listing,plus,active,for_sale,-7.0,gamla_stan,20221217,True,False,False,False,203 m²,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18821279,"[59.338731747003905, 18.035011132523685]","Bråvallagatan 8, 5 tr",Stockholms kommun,Bostadsrättslägenhet,False,False,3 500 000 kr,134 615 kr/m²,True,1 rum,2 230 kr/mån,114,False,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-1rum-bir...,[],0,Ambassadör Fastighetsmäkleri AB,"Birkastan, Stockholms kommun",False,18821279,common_listing,basic,active,for_sale,17.0,vasastan,20221217,False,True,False,False,26 m²,None
18991743,"[59.34305, 18.0319862]",Väringgatan 27,Stockholms kommun,Bostadsrättslägenhet,False,False,3 500 000 kr,109 375 kr/m²,Tru

In [21]:
#remove weird whitespace strings
df2 = df2.replace('\xa0', '', regex=True)

import re

strToIntList=['fee','price','price_per_area', 'living_area']
df[strToIntList].fillna(value=0, inplace=True)

for c in strToIntList:
    df2[c]= re.sub(r'kr/mån|fr.|kr| |kr/m²', "", df2[c])

In [9]:
#remove and replace some unwanted things in listed columns
strToIntList=['fee','price','price_per_area', 'living_area']
i=2

df2[strToIntList].fillna(value=np.nan, inplace=True)

for c in strToIntList:
    df2[c]=df2[c].str.strip()
    df2[c]=df2[c].str.replace('kr/mån','')
    df2[c]=df2[c].str.replace('kr/m²','')
    df2[c]=df2[c].str.replace('fr.','',regex=False)
    df2[c]=df2[c].str.replace('kr','')
    df2[c]=df2[c].str.replace(' ', '')
    df2[c]=df2[c].str.replace('/', '')
    df2[c]=df2[c].str.replace('m²','')
    #df2.fee.fillna(value=np.nan, inplace=True)
    df2[c]=df2[c].fillna(0)
#df2['price_per_area']=df2['price_per_area'].str.replace('kr/m²','').str.replace('fr.','')

#
#

/var/folders/l0/yr9js0tn3m3dc3_slvzvwsgc0000gn/T/ipykernel_8995/2819892623.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[strToIntList].fillna(value=np.nan, inplace=True)


In [10]:
#other str replaces
df2['rooms']=df2['rooms'].str.replace(' rum','')
df2['price']=df2['price'].replace('Prissaknas', 0)

In [11]:
df2['fee']=df2['fee'].astype(int)
df2['price']=df2['price'].astype(int)
df2['price_per_area']=df2['price'].astype(int)

In [35]:
df2['living_area']=df2['living_area'].replace(',', '.')
tt = str.maketrans(',', '.', '.€%')
df2.living_area = df2.living_area.str.translate(tt)
df2['living_area']

id
19324990     127
19325173      64
19102415     151
18956296    98.5
19158120     175
            ... 
18987175    30.5
18976122      56
18941998     124
18669396      34
16998076      34
Name: living_area, Length: 1263, dtype: object

----- will drop these ones for now, since they are very weird in general... ------

In [12]:
todrop=df2.loc[df2['living_area'].str.contains('–',na=False)]
df3=df2.drop(index=todrop.index)

In [13]:
df3.loc[df3['living_area'].str.contains('–',na=False)]

,coordinate,address,location_name,typeSummary,ongoing_bidding,foreclosure,price,price_per_area,has_price_change,rooms,...,trackingData.listingStatus,price_change,area,date,balcony,elevator,patio,tenure_owned,living_area,supplemental_area
id,,,,,,,,,,,,,,,,,,,,,


In [14]:
df3['living_area']=df3['living_area'].astype('float')

ValueError: could not convert string to float: '35,5'

In [63]:
df3.to_parquet('../data/2nd_transformed_data.parquet')